In [14]:
!pip install numpy pandas tensorflow scikit-learn
!pip uninstall wandb -y
!pip install wandb

Found existing installation: wandb 0.18.0
Uninstalling wandb-0.18.0:
  Successfully uninstalled wandb-0.18.0
  Using cached wandb-0.18.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached wandb-0.18.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)


In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import wandb
from wandb.keras import WandbCallback

# Prepare the data
- Load dataset from CSV
- clean unnecessary columns: 'LastName', 'FirstName', and 'PhoneNumber'
- Separate features (X) and target (y)

In [17]:

data = pd.read_csv('loaners-repayment-data.csv')

data = data.drop(columns=['LastName', 'FirstName', 'PhoneNumber'])

X = data[['Age', 'Income', 'DebtAmount', 'CreditScore', 'LoanDuration', 'MissedPayments']]
y = data['Repay']


# Split the data into training and testing sets

In [18]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Normalize the input features

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Initialize Wandb

In [20]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
wandb.init(project="Chapter5-loaners-repayment-data", entity="yasno-llc")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yasnosoft (yasno-llc). Use `wandb login --relogin` to force relogin


# Build the neural network model

In [22]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Input layer specifying the input shape
    Dense(32, activation='relu'),      # First hidden layer with 32 neurons
    Dense(64, activation='relu'),      # Second hidden layer with 64 neurons
    Dense(1, activation='sigmoid')     # Output layer with 1 neuron (for binary classification)
])

# Compile the model

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

In [24]:
# Manually log metrics during training
epochs = 50
batch_size = 10
for epoch in range(epochs):
    # Train the model for one epoch
    history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)

    # Get training metrics
    train_loss = history.history['loss'][0]
    train_accuracy = history.history['accuracy'][0]

    # Log training metrics to wandb
    wandb.log({"Epoch": epoch + 1, "Train Loss": train_loss, "Train Accuracy": train_accuracy})

    print(f'Epoch {epoch + 1}/{epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')


Epoch 1/50 - Loss: 0.7000, Accuracy: 0.4688
Epoch 2/50 - Loss: 0.6894, Accuracy: 0.5375
Epoch 3/50 - Loss: 0.6832, Accuracy: 0.5688
Epoch 4/50 - Loss: 0.6805, Accuracy: 0.5562
Epoch 5/50 - Loss: 0.6714, Accuracy: 0.6000
Epoch 6/50 - Loss: 0.6690, Accuracy: 0.6125
Epoch 7/50 - Loss: 0.6673, Accuracy: 0.6125
Epoch 8/50 - Loss: 0.6588, Accuracy: 0.6438
Epoch 9/50 - Loss: 0.6578, Accuracy: 0.6438
Epoch 10/50 - Loss: 0.6518, Accuracy: 0.6250
Epoch 11/50 - Loss: 0.6479, Accuracy: 0.6187
Epoch 12/50 - Loss: 0.6428, Accuracy: 0.6625
Epoch 13/50 - Loss: 0.6378, Accuracy: 0.6500
Epoch 14/50 - Loss: 0.6348, Accuracy: 0.6625
Epoch 15/50 - Loss: 0.6294, Accuracy: 0.6562
Epoch 16/50 - Loss: 0.6259, Accuracy: 0.6625
Epoch 17/50 - Loss: 0.6209, Accuracy: 0.6750
Epoch 18/50 - Loss: 0.6149, Accuracy: 0.6812
Epoch 19/50 - Loss: 0.6099, Accuracy: 0.6938
Epoch 20/50 - Loss: 0.6047, Accuracy: 0.6938
Epoch 21/50 - Loss: 0.6006, Accuracy: 0.7063
Epoch 22/50 - Loss: 0.5963, Accuracy: 0.6938
Epoch 23/50 - Loss:

# Evaluate the model on the test set

In [25]:

test_loss, test_acc = model.evaluate(X_test, y_test)

print(f'Test Accuracy: {test_acc}')

# Log the final test accuracy to wandb
wandb.log({"Test Accuracy": test_acc})

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5271 - loss: 0.8362 
Test Accuracy: 0.5249999761581421
